In [1]:
import numpy as np 
import pandas as pd
from docplex.mp.model import Model
from docplex.mp.solution import SolveSolution
from typing import Union, List
from numpy.random import default_rng
import names

from datetime import datetime
rng = default_rng()

## Constants

In [2]:
number_exams = np.random.randint(5, 10) 
number_students = np.random.randint(50, 60)
number_exam_days = np.random.randint(7, 15) # number of days allocated for final exams
number_rooms = np.random.randint(5, 10) 

E = [f'CSC_{code}' for code in rng.choice(range(101, 401), size=number_exams, replace=False)] # exams
S = [names.get_full_name() for i in range(number_students)] # students
T = [[datetime(2022, 12, day, 9), datetime(2022, 12, day, 13), datetime(2022, 12, day, 17)] for day in range(1, number_exam_days)] # timeslots
R = [f'RM_{code}' for code in rng.choice(range(101, 401), size=number_exams, replace=False)] # rooms
Cp = np.random.randint(20, 60, (len(R))) # capacity of rooms

# course enrolments
He_s = np.random.randint(0,2, (len(E),len(S))) # binary random 
sumHe_s = np.sum(He_s, axis=1)

## Variables

In [3]:
opt_mod = Model(name='ET MIP')

In [4]:
x = opt_mod.binary_var_matrix(len(E), len(T), name="X_e,t") # whether we use timeslot t for exam e
y = opt_mod.binary_var_matrix(len(E), len(R), name="Y_e,r") # whether we use room r for exam e
# z = opt_mod.binary_var_matrix(len(S), len(E), name="Z_s,e") # whether exam e is allocated to student s 

## Constraints

C1: For all exams, the sum of the allocated timeslots must be equal to 1

$$\sum_{t\in T_c} X_e,_t=1 \;\forall \; e \in E, e \in \{0,1\}$$

C2: For all exams, the sum of the allocated rooms must be equal to 1

$$\sum_{r\in R_e} Y_e,_r = 1 \;\forall \; e \in E $$

C3: For every student and timeslot, the sum of the allocated exams must be less or equal to 1. 
- i.e. students can be at only one exam at a time

C4: For all rooms, the sum of students in a room must be less than the capacity of the room

In [5]:
c1 = opt_mod.add_constraints((sum(x[e, t] for t in range(len(T))) == 1 for e in range(len(E))), names='c1') 

In [6]:
c2 = opt_mod.add_constraints((sum(y[e, r] for r in range(len(R))) == 1 for e in range(len(E))), names='c2') 

In [7]:
# c3 modified constraint 
for s in range(len(S)):
    for t in range(len(T)):
        cond = sum(x[e,t] * He_s[e,s] for e in range(len(E)))
        if type(cond) != int:
            opt_mod.add_constraint(cond <= 1)

In [8]:
# c4 modified constraint
for r in range(len(R)):
    for t in range(len(T)):
        cond = sum((x[e,t]*y[e,r]) * sumHe_s[e] for e in range(len(E)))
        if type(cond) != int:
            opt_mod.add_constraint(cond <= Cp[r])

In [9]:
# c4 = opt_mod.add_constraints((sum(y[e,r] * sumHe_s[e] for e in range(len(E))) <= Cp[r] for r in range(len(R))), names='c4') 

## Objective Function

$$  minimize\; I_T = \sum_{k=1}^{K_T} \; ceil \; \left[ \sum_{c=1}^{C_K}\; N_c \; * \; (ratio \; students \; to \; invigilators) \right] $$

In [10]:
ratio_of_Inv = 1/3

In [11]:
obj_fun =  sum(sum(y[e,r] * sumHe_s[e] for e in range(len(E))) for r in range(len(R)))
opt_mod.set_objective('min', obj_fun)
opt_mod.print_information()

Model: ET MIP
 - number of variables: 65
   - binary=65, integer=0, continuous=0
 - number of constraints: 482
   - linear=442, quadratic=40
 - parameters: defaults
 - objective: minimize
 - problem type is: MIQCP


## Processing Solution

In [12]:
def process_solution(sol : SolveSolution) -> Union[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Takes a cplex solution and produces a exam schedule
    
    Parameters
    ----------
    sol : SolveSolution
        solution from the solver
    
    Returns
    -------
    final_schedule : pd.DataFrame
        The schedule formatted in readable format for an exam organizer
    
    df_x : pd.DataFrame
        The results for variable x
    
    df_y : pd.DataFrame
        The results for variable y
    """
    # extract solutions as df
    df_x = sol.get_value_df(x).rename(columns={'key_1':'exam','key_2':'timeslot'})
    df_y = sol.get_value_df(y).rename(columns={'key_1':'exam','key_2':'room'})

    # Add rows with the names of courses and timelots
    exam_col = [E[i] for i in range(len(E)) for j in range(len(T))]
    time_col = [T[j] for i in range(len(E)) for j in range(len(T))]
    df_x["EXAM"] = exam_col
    df_x["TIMESLOT"] = time_col

    # Add rows with the names of courses and rooms
    exam_col = [E[i] for i in range(len(E)) for j in range(len(R))]
    room_col = [R[j] for i in range(len(E)) for j in range(len(R))]
    df_y["EXAM"] = exam_col
    df_y["ROOM"] = room_col
    
    # Produce the final schedule
    final_schedule = df_x[df_x["value"]==1].merge(df_y[df_y["value"]==1], on='EXAM', how='left')
    
    return final_schedule, df_x, df_y

In [13]:
def create_enrolment_df(He_s : np.array, S : List[int]) -> pd.DataFrame:
    """
    Creates a dataframe with the students for each exam/course
    """
    exam_student_pairs = []
    for exam in range(len(He_s)):
        students_in_exam_e = []
        for i, student in enumerate(He_s[exam]):
            if student == 1:
                students_in_exam_e.append(S[i])
        exam_student_pairs.append(students_in_exam_e)
        
    enrolment_df = pd.DataFrame(columns=['EXAM','student'])
    enrolment_df['EXAM'] = E
    enrolment_df['student'] = exam_student_pairs

    return enrolment_df

In [14]:
enrolment_df = create_enrolment_df(He_s, S)
enrolment_df

,EXAM,student
0,CSC_190,"[Dan Toler, Consuelo Valencia, Oscar Obrien, K..."
1,CSC_338,"[Dan Toler, Bobby Powell, Joyce Brown, Glenda ..."
2,CSC_231,"[Dan Toler, Bobby Powell, Maryanne Jones, Robe..."
3,CSC_302,"[Richard Noga, Bobby Powell, Consuelo Valencia..."
4,CSC_138,"[Dan Toler, Richard Noga, Bobby Powell, Oscar ..."


In [15]:
sol = opt_mod.solve()
if sol:
    print("Found a solution \n")
    schedule, df_x, df_y = process_solution(sol)
    print("Schedule: \n")
    display(schedule.merge(enrolment_df, on='EXAM', how='left'))
    
    run_time = sol.solve_details.time
    
else:
    print("Could not find a solution")
    run_time = np.nan

Found a solution 

Schedule: 



,exam_x,timeslot,value_x,EXAM,TIMESLOT,exam_y,room,value_y,ROOM,student
0,0,1,1.0,CSC_190,"[2022-12-02 09:00:00, 2022-12-02 13:00:00, 202...",0,2,1.0,RM_102,"[Dan Toler, Consuelo Valencia, Oscar Obrien, K..."
1,1,5,1.0,CSC_338,"[2022-12-06 09:00:00, 2022-12-06 13:00:00, 202...",1,3,1.0,RM_136,"[Dan Toler, Bobby Powell, Joyce Brown, Glenda ..."
2,2,0,1.0,CSC_231,"[2022-12-01 09:00:00, 2022-12-01 13:00:00, 202...",2,1,1.0,RM_193,"[Dan Toler, Bobby Powell, Maryanne Jones, Robe..."
3,3,4,1.0,CSC_302,"[2022-12-05 09:00:00, 2022-12-05 13:00:00, 202...",3,2,1.0,RM_102,"[Richard Noga, Bobby Powell, Consuelo Valencia..."
4,4,2,1.0,CSC_138,"[2022-12-03 09:00:00, 2022-12-03 13:00:00, 202...",4,3,1.0,RM_136,"[Dan Toler, Richard Noga, Bobby Powell, Oscar ..."
